<a href="https://colab.research.google.com/github/renuchaurasia/renu_INFO5731_Spring2023/blob/main/Assignment/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [1]:
# opening the file in read mode
my_file = open("/content/sample_data/WebListWalmart.txt", "r")
  
# reading the file
data = my_file.read()

data_into_list = data.replace('"', '')
data_into_list = data_into_list.replace('\n', ' ').split(",")
  
# printing the data
# print(data_into_list)
my_file.close()

In [3]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.3 MB/s eta 0:00:00


In [1]:
# You code here (Please add comments in the code):

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
from IPython.display import display
import time

# Product page URL (Put your URL)
url_list = ["https://www.walmart.com/ip/Super-Mario-Kid-s-Spinbrush-Electric-Battery-Toothbrush-Soft-1-ct-Character-May-Vary/35790405"]
# data_into_list
reviews =[]

for url in url_list:
    # Get Product Id
    product_id = url[url.rfind("/")+1::]
    int_url = url[0:url.rfind("/")]
    product_name = int_url[int_url.rfind("/")+1::].replace("-"," ")
    header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36','referer':'https://www.amazon.com/s?k=nike+shoes+men&crid=28WRS5SFLWWZ6&sprefix=nike%2Caps%2C357&ref=nb_sb_ss_organic-diversity_2_4'}
    response = requests.get("https://www.walmart.com/reviews/product/"+product_id, headers=header)
    print(response.content)
    if "Activate and hold the button to confirm that you" not in str(response.content):
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("li",{'class':"dib w-100 mb3"}):
            review = {}
            stars = i.findAll('span', {'class':"w_DF"})
            heading = i.findAll('span', {'class':"b w_O"})
            review_text = i.findAll('span', {'class':"tl-m db-m"})
            reviewer = i.findAll('div', {'class':"f6 gray pr2"})
            review_date = i.findAll('div', {'class':"f7 gray"})

            review["product_name"] = product_name
            review["reviewer"] = reviewer[0].text if len(reviewer)>0 else ""
            review["review_date"] = review_date[0].text
            review["review_text"] = review_text[0].text

            reviews.append(review)
            
    else:
        print("Robort Error")

    time.sleep(10)
# display(pd.DataFrame(reviews))
pd.DataFrame(reviews).to_csv('Walmart_product_review.csv', sep=',')

b'<html lang="en">\n\n<head>\n    <title>Robot or human?</title>\n    <meta name="viewport" content="width=device-width">\n    <style>\n    #sign-in-widget a,\n    #sign-in-widget a:active,\n    #sign-in-widget a:hover {\n        color: #000\n    }\n\n    #sign-in-widget h1 {\n        font-weight: 500;\n        font-size: 20px;\n        font-size: 1.25rem;\n        letter-spacing: -.6px;\n        margin: 1px auto\n    }\n\n    @media (min-width:30em) {\n        #sign-in-widget h1 {\n            margin-top: 24px;\n            font-size: 24px;\n            font-size: 1.5rem\n        }\n    }\n\n    #sign-in-widget {\n        font-family: BogleWeb, Helvetica Neue, Helvetica, Arial, sans-serif\n    }\n\n    #sign-in-widget * {\n        box-sizing: border-box\n    }\n\n    #sign-in-widget .text-right {\n        text-align: right\n    }\n\n    @font-face {\n        font-family: NewYorkIcons;\n        src: url(6255ed72d86ece856725a2d80878bce6.eot);\n        font-weight: 400;\n        font-sty

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [3]:
# Write your code here
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
from IPython.display import display

df_walmart_product_review = pd.read_csv('Walmart_product_review.csv')
df_walmart_product_review.columns = ['Sno','product_name', 'reviewer', 'review_date', 'review_text']

#Removing noise, such as special characters and punctuations.
df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text'].str.replace('[^\w\s]','')


#Remove numbers
df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text_cln'].str.replace('\d+', '').astype(str)

df_walmart_product_review.head()


<ipython-input-3-6986a8814e7a>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text'].str.replace('[^\w\s]','')
<ipython-input-3-6986a8814e7a>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text_cln'].str.replace('\d+', '').astype(str)


,Sno,product_name,reviewer,review_date,review_text,review_text_cln
0,0,Melrose Ave Women s Faux Leather Crossband Sli...,HilC,3/3/2022,These sandals look just like the picture! Grea...,These sandals look just like the picture Great...
1,1,Melrose Ave Women s Faux Leather Crossband Sli...,Jojo,3/3/2022,"These slides feel great on and they look nice,...",These slides feel great on and they look nice ...
2,2,Melrose Ave Women s Faux Leather Crossband Sli...,MaryjoB02,3/3/2022,"Fit is great: comfortable, no catching of toes...",Fit is great comfortable no catching of toes p...
3,3,Melrose Ave Women s Faux Leather Crossband Sli...,Karenc,3/3/2022,"These sandals are so pretty, so comfortable an...",These sandals are so pretty so comfortable and...
4,4,Melrose Ave Women s Faux Leather Crossband Sli...,Cocochies,3/6/2022,Melrose Ave Vegan Leather Crossband Slide Sand...,Melrose Ave Vegan Leather Crossband Slide Sand...


In [12]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:
#Removing stopwords
from nltk.corpus import stopwords

stop = stopwords.words('english')
df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text_cln'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df_walmart_product_review['review_text_cln'].head()

0    These sandals look like picture Great hotter m...
1    These slides feel great look nice I really lov...
2    Fit great comfortable catching toes padded foo...
3    These sandals pretty comfortable fit perfectly...
4    Melrose Ave Vegan Leather Crossband Slide Sand...
Name: review_text_cln, dtype: object

In [8]:
result = df_walmart_product_review.dtypes
print(result)

Sno                 int64
product_name       object
reviewer           object
review_date        object
review_text        object
review_text_cln    object
dtype: object


In [9]:
#Lower case
df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text_cln'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_walmart_product_review['review_text_cln'].head()

0    these sandals look like picture great hotter m...
1    these slides feel great look nice i really lov...
2    fit great comfortable catching toes padded foo...
3    these sandals pretty comfortable fit perfectly...
4    melrose ave vegan leather crossband slide sand...
Name: review_text_cln, dtype: object

In [10]:
#Stemming. 

from nltk.stem import PorterStemmer
st = PorterStemmer()
df_walmart_product_review['review_text_cln'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    these sandal look like pictur great hotter mon...
1    these slide feel great look nice i realli love...
2    fit great comfort catch toe pad footb stay wit...
3    these sandal pretti comfort fit perfectli they...
4    melros ave vegan leather crossband slide sanda...
Name: review_text_cln, dtype: object

In [13]:
#Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')

df_walmart_product_review['review_text_cln'] = df_walmart_product_review['review_text_cln'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df_walmart_product_review['review_text_cln'].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    these sandal look like picture great hotter mo...
1    these slide feel great look nice i really love...
2    fit great comfortable catching toe padded foot...
3    these sandal pretty comfortable fit perfectly ...
4    melrose ave vegan leather crossband slide sand...
Name: review_text_cln, dtype: object

In [14]:
df_walmart_product_review

,Sno,product_name,reviewer,review_date,review_text,review_text_cln
0,0,Melrose Ave Women s Faux Leather Crossband Sli...,HilC,3/3/2022,These sandals look just like the picture! Grea...,these sandal look like picture great hotter mo...
1,1,Melrose Ave Women s Faux Leather Crossband Sli...,Jojo,3/3/2022,"These slides feel great on and they look nice,...",these slide feel great look nice i really love...
2,2,Melrose Ave Women s Faux Leather Crossband Sli...,MaryjoB02,3/3/2022,"Fit is great: comfortable, no catching of toes...",fit great comfortable catching toe padded foot...
3,3,Melrose Ave Women s Faux Leather Crossband Sli...,Karenc,3/3/2022,"These sandals are so pretty, so comfortable an...",these sandal pretty comfortable fit perfectly ...
4,4,Melrose Ave Women s Faux Leather Crossband Sli...,Cocochies,3/6/2022,Melrose Ave Vegan Leather Crossband Slide Sand...,melrose ave vegan leather crossband slide sand...
...,...,...,...,...,...,...
466,466,REESE S Milk Chocolate Peanut Butter Eggs Cand...,Tina,4/14/2020,ALL the Reese's eggs were melted and misshapen...,all reeses egg melted misshapen i received
467,467,REESE S Milk Chocolate Peanut Butter Eggs Cand...,snakemom,3/21/2017,The candy was much fresher than the candy in t...,the candy much fresher candy store detroit
468,468,REESE S Milk Chocolate Peanut Butter Eggs Cand...,knousey,3/6/2017,i think these are the best peanut butter eggs ...,think best peanut butter egg even better homem...
469,469,REESE S Milk Chocolate Peanut Butter Eggs Cand...,Nutties,3/12/2017,Great tasting chocolate with a yummy peanut bu...,great tasting chocolate yummy peanut butter ce...


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [21]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [22]:
#3-1
import nltk
from collections import Counter

import spacy

# Tokenize the text into words
df_walmart_product_review['review_text'] = df_walmart_product_review['review_text'].astype(str)
df_walmart_product_review['words'] = df_walmart_product_review['review_text'].apply(nltk.word_tokenize)

# Perform POS tagging for each row
df_walmart_product_review['pos_tags'] = df_walmart_product_review['words'].apply(nltk.pos_tag)

# Count the number of each POS tag for each row
df_walmart_product_review['pos_counts'] = df_walmart_product_review['pos_tags'].apply(lambda x: Counter(tag[1][0] for tag in x))

# Sum up the total number of each POS tag across all rows
total_counts = df_walmart_product_review['pos_counts'].sum()

# Print the results
print("Total number of nouns (N):", total_counts['N'])
print("Total number of verbs (V):", total_counts['V'])
print("Total number of adjectives (Adj):", total_counts['J'])
print("Total number of adverbs (Adv):", total_counts['R'])

Total number of nouns (N): 5335
Total number of verbs (V): 4192
Total number of adjectives (Adj): 1940
Total number of adverbs (Adv): 1624


In [43]:
#3-2
import spacy
from spacy import displacy
nlp=spacy.load('en_core_web_sm')

for i in df_walmart_product_review['review_text'].head():
  displacy.render(nlp(i),jupyter=True)

In [44]:
from nltk import pos_tag, word_tokenize, RegexpParser

chunker = RegexpParser(""" 
                       NP: {<DT>?<JJ>*<NN>}    #To extract Noun Phrases 
                       P: {<IN>}               #To extract Prepositions 
                       V: {<V.*>}              #To extract Verbs 
                       PP: {<P> <NP>}          #To extract Prepostional Phrases 
                       VP: {<V> <NP|PP>*}      #To extarct Verb Phrases 
                       """)
for i in df_walmart_product_review['pos_tags']:
  output = chunker.parse(i)
  print("After Extracting\n", output) 

Streaming output truncated to the last 5000 lines.
After Extracting
 (S
  (NP Yummmyyy/NN)
  !/.
  They/PRP
  (VP (V were/VBD))
  fresh/JJ
  and/CC
  delicious/JJ
  ./.)
After Extracting
 (S
  (NP [/NN)
  (NP This/DT review/NN)
  (VP (V was/VBD))
  (VP
    (V collected/VBN)
    (PP (P as/IN) (NP part/NN))
    (PP (P of/IN) (NP a/DT promotion/NN)))
  ./.
  (NP ]/NN)
  I/PRP
  (VP (V love/VBP) (NP springtime.The/JJ weather/NN))
  (VP (V is/VBZ))
  beautiful/JJ
  and/CC
  the/DT
  flowers/NNS
  (VP (V are/VBP))
  (VP (V blooming/VBG))
  ./.
  The/DT
  Easter/NNP
  Bunny/NNP
  (VP (V is/VBZ) (PP (P on/IN) (NP the/DT way/NN)))
  to/TO
  (VP (V visit/VB) (NP every/DT bunny/NN))
  and/CC
  REESE/NNP
  'S/POS
  Peanut/NNP
  Butter/NNP
  Eggs/NNP
  (VP (V appear/VBP) (PP (P on/IN) (NP store/NN)))
  shelves/NNS
  ./.
  They/PRP
  (VP (V are/VBP))
  fresh/JJ
  and/CC
  delicious/JJ
  !/.
  They/PRP
  never/RB
  (VP (V stick/VBP))
  around/RP
  long/RB
  ./.
  Get/NNP
  em/RB
  (P while/IN)
  you/

In [37]:
#3-3
#) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and 
# date from the clean texts, calculate the count of each entity.
import spacy

import en_core_web_sm
nlp = en_core_web_sm.load()

entity_types = ['PERSON','NORP','FAC','ORG','GPE','LOC','PRODUCT','EVENT','WORK_OF_ART','LAW',
                'LANGUAGE','DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']

entity_name = ['person','Nationality','Building','Institution','country','location','PRODUCT','EVENT','Title','LAW',
                'LANGUAGE','DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']

def entity_extraction(sentence):
    entity_list = []
    doc = nlp(sentence)
    for ent in doc.ents:
        entity_list.append([ent.text, ent.start_char, ent.end_char, ent.label_])
    
    return entity_list

for text in df_walmart_product_review['review_text']:

  print(entity_extraction(text))


[['the hotter months', 52, 69, 'DATE']]
[]
[]
[]
[['summer', 73, 79, 'DATE']]
[['6', 86, 87, 'CARDINAL'], ['half', 213, 217, 'CARDINAL']]
[['Melrose', 18, 25, 'NORP'], ['spring', 102, 108, 'DATE'], ['summer', 113, 119, 'DATE']]
[['Florida', 53, 60, 'GPE'], ['Capri', 112, 117, 'ORG'], ['first', 379, 384, 'ORDINAL'], ['one', 446, 449, 'CARDINAL'], ['a few short hours', 602, 619, 'TIME'], ['5', 677, 678, 'CARDINAL'], ['3', 698, 699, 'CARDINAL']]
[]
[['first', 14, 19, 'ORDINAL']]
[]
[]
[]
[]
[]
[]
[['the spring', 78, 88, 'DATE']]
[['the end of the day', 85, 103, 'DATE']]
[['Color', 97, 102, 'ORG']]
[['this summer', 26, 37, 'DATE']]
[['EXTREMELY', 64, 73, 'ORG'], ['Steve Madden', 89, 101, 'PERSON'], ['Pinterest', 130, 139, 'GPE'], ['Instagram', 144, 153, 'ORG'], ['89.99', 179, 184, 'MONEY'], ['almost half', 314, 325, 'CARDINAL']]
[['Amazon', 55, 61, 'ORG'], ['WalMart', 139, 146, 'ORG']]
[['1/2', 69, 72, 'CARDINAL'], ['One', 85, 88, 'CARDINAL']]
[]
[['$41 to $59', 187, 197, 'MONEY'], ['less 

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency parsing tree:

Constituency parsing refers to the technique of examining the grammatical structure of a sentence to recognize the units of words that act as a group within the sentence. These units are called constituents. A constituency parsing tree represents this process visually, displaying the hierarchical connections between the constituents of the sentence.

"John ate a sandwich for lunch."

To construct a constituency parsing tree for this sentence, we first identify its constituent parts. In this case, the sentence consists of a noun phrase ("John") and a verb phrase ("ate a sandwich for lunch").

The noun phrase can be further broken down into a proper noun ("John"), while the verb phrase can be broken down into a verb ("ate") and a noun phrase ("a sandwich for lunch"). The noun phrase consists of a determiner ("a") and a noun ("sandwich"), while the prepositional phrase ("for lunch") consists of a preposition ("for") and a noun phrase ("lunch").




Dependency parsing tree:

Dependency parsing is a technique that identifies the relationships between words in a sentence based on their syntactic dependencies. A dependency parsing tree is a graphical representation of these dependencies, with each word represented as a node, and directed arcs indicating the dependencies between them. This technique is useful for various natural language processing tasks such as text-to-speech conversion and machine translation.

example: "These sandals look just like the picture! Great for the hotter months."

The tree starts with the root word "look", which is the main verb of the sentence. The subject of this verb is "sandals".

The phrase "just like the picture" modifies the verb "look" and is represented as a prepositional phrase with the preposition "like" and the object "picture". The adjective "just" modifies the prepositional phrase.

The second sentence starts with the word "Great" as the main verb, with the object "months". The phrase "for the hotter months" is a prepositional phrase modifying the verb "Great", with the preposition "for" and the object "months".

The tree provides a graphical representation of these relationships, making it easier to identify the dependencies between words and understand the structure of the sentence.
